# Converting a dataset

Run to execute
Use Restart to run again

You'll be asked to select an Excel file that contains the mapping and configuration

In [1]:
# Run to execute, use Restart to run again

import json
import os
from datetime import datetime

import numpy as np
import pandas as pd


# some functions to allow to create more than one conversion.xlsx

class StopExecution(Exception):
    def _render_traceback_(self):
        pass


def get_all_items(folder, type="folders"):
    """
    This function gets by default all the subfolders from folder
    If type == files, then it gets all the files in that folder
    """
    all_outputs = []
    for item in os.listdir(folder):
        if type == "folders":
            if os.path.isdir(folder + "/" + item):
                all_outputs.append(item)
        if type == "files":
            if os.path.isfile(folder + "/" + item):
                all_outputs.append(item)
    return sorted(all_outputs, reverse=True)


def title_wrapper(func):
    """
    This function wraps the title with * and -.
    
    Used by print_title
    """

    def wrapper(*args, **kwargs):
        print("*" * 90)
        func(*args, **kwargs)
        print("-" * 90)

    return wrapper


@title_wrapper
def print_title(text):
    """
    This function wraps the text in the title_wrapper
    """
    print(text)


def choose_retry(text):
    """
    This generic function returns the input from the user.
    """
    return input("Please choose Q(uit) or between " + text + ": ")


def choose_dir_item(folder, type="folders", what="All"):
    """
    Pick a subfolder or file from a folder.
    
    """
    all_items = get_all_items(folder, type)
    print_title("Choose source by number")
    # create dictionary with all folders
    items = {}
    if what != "All":
        temp_all_items = []
        for item in all_items:
            if what in item:
                temp_all_items.append(item)
        all_items = temp_all_items

    for count, item in enumerate(all_items):
        items[count + 1] = item
        extra = ""
        if count + 1 < 10:
            extra = " "
        print(f"[{count+1}]{extra} {items[count+1]}", end="\t")
        if (count + 1) % 3 == 0:
            print("")
    print("\n")
    item_choosen = False
    while item_choosen == False:
        this_answer = choose_retry(str(1) + " and " + str(count + 1))
        try:
            if int(this_answer) > 0 and int(this_answer) <= count + 1:
                item_choosen = True
                return all_items[int(this_answer) - 1]
        except:
            if this_answer.lower() == "q":
                print_title("Quiting the notebook run on instruction of the user")
                item_choosen = True
                raise StopExecution
                
                # select the conversion.xslx
folder = "./"
print("\n" * 2)
print_title("Select the Excel containing all the configuration & mapping")
conversion_excel = choose_dir_item(folder, "files", "xlsx")


# read converstion table
# conversion_excel = 'conversion.xlsx'
sheet_name = "Conversion_Table"
conversion_table = pd.read_excel(
    conversion_excel, sheet_name=sheet_name, index_col=None
)
sheet_name = "Key_Table"
key_table = pd.read_excel(
    conversion_excel,
    sheet_name=sheet_name,
    index_col="Key_Old",
    converters={"New_Key": str, "Old_Key": str},
)
sheet_name = "Settings"
settings_table = pd.read_excel(
    conversion_excel, sheet_name=sheet_name, index_col="Item"
)

# read settings
source_dir = settings_table.loc["source_dir"]["Variable"]
converted_dir = settings_table.loc["converted_dir"]["Variable"]
source_file = settings_table.loc["source_filename"]["Variable"]
source_separator = settings_table.loc["source_separator"]["Variable"]
converted_file = settings_table.loc["converted_filename"]["Variable"]
converted_separator = settings_table.loc["converted_separator"]["Variable"]

# read data
df = pd.read_csv(source_dir + "/" + source_file, index_col=None)

# go through the conversion table to make the changes
for row in conversion_table.iterrows():
    # reading the necessary variables
    nv = row[1]["New_Variable"]
    toc = row[1]["TypeOfConversion"]
    mv = row[1]["Map_Variable"]
    con = row[1]["Conversion"]

    # Only do stuff when there is a mapping
    if mv == mv:
        # convert the conversion to a dictionary
        if con == con and str(con)[0] == "{":
            res = json.loads(con)

        # convert to lower case just in case of typo
        conversion_type = str(toc).lower()

        # normal variable swapping with value conversion
        if conversion_type == "normal":
            df = df.rename(columns={mv: nv})
            # convert values if con is not empty
            if con == con:
                # convert dtype of target because dictionary expects a string
                df[nv] = df[nv].astype("string")
                # make sure if a value is missed, that it is visible
                df[nv] = df[nv].map(res).fillna("mapping missing")

        if conversion_type == "check2option" and con == con:
            # create empty list of all the checkbox variables
            subcolumns = []
            for key, value in res.items():
                checkbox = mv + "#" + key
                subcolumns.append(checkbox)
                # assign right value to the checkbox column
                df[checkbox] = df[checkbox].map({1: value})
            # join the checkbox columns into target column
            df[nv] = df[subcolumns].apply(
                lambda x: ",".join(x.dropna().astype(str)), axis=1
            )
            # remmve all the check box columns
            df.drop(subcolumns, axis=1, inplace=True)

        # convesion of options to checkbox format
        if conversion_type == "option2check" and con == con:
            # convert dtype of target because dictionary expects a string
            df[mv] = df[mv].astype("string")
            # counter is a counter that keeps track of the value to be expected
            counter = 1
            # create the checkbox variable and add the value '1' if applicable
            for key, value in res.items():
                # new checkbox variable
                checkbox = nv + "#" + value
                # add the values of the old variable
                df[checkbox] = df[mv]
                # add the value '1' if correct, leave empty otherwise
                df[checkbox] = df[checkbox].map({str(counter): "1"}).fillna("")
                counter += 1
            # delete the old column
            del df[mv]

        # adding units
        if conversion_type == "unit":
            # copy the values
            df[nv] = df[mv]
            # add the units when not empty
            if df[nv].dtype == "O":
                df.loc[df[nv] != "", nv] = str(con)
            else:
                df.loc[df[nv].notnull(), nv] = str(con)
                df[nv].replace(np.nan, "", regex=True, inplace=True)

        # replacing keys
        if conversion_type == "id":
            # copy the values
            df[nv] = df[mv]
            # convert key_table into dictionary
            res = key_table.to_dict("dict")
            res = {str(key): str(value) for key, value in res["Key_New"].items()}
            df[nv] = df[nv].astype("string")
            # make sure if a value is missed, that it is visible
            df[nv] = df[nv].map(res).fillna("mapping missing")


# Setting the correct order, repeating the loop, but easier and more robust to do it here
# ordered list of columns
column_order = []
for row in conversion_table.iterrows():
    # reading the necessary variables
    nv = row[1]["New_Variable"]
    toc = row[1]["TypeOfConversion"]
    con = row[1]["Conversion"]
    conversion_type = str(toc).lower()

    # add missing variables except when conversion_type == option2check
    if nv not in df.columns and conversion_type != "option2check":
        df[nv] = ""
        column_order.append(nv)

    # add missing variables for conversion_type == option2check, requirement is that there is a mapping
    elif con == con and str(con)[0] == "{" and conversion_type == "option2check":
        res = json.loads(con)
        for key, value in res.items():
            # new checkbox variable
            checkbox = nv + "#" + value
            column_order.append(checkbox)
    else:
        # in all other cases add the variable to the list
        column_order.append(nv)


# sort the dataframe in the correct order
df = df[column_order]


## write converted data
file_name = converted_dir + f"/{datetime.now():%Y%m%d-%H%M%S}_" + converted_file
df.to_csv(file_name + ".csv", sep=converted_separator, index=False)
df.to_excel(file_name + ".xlsx", index=False)

print_title(
    f"All done, the output (cSV and Excel) can be found timestamped in: {converted_dir}"
)




******************************************************************************************
Select the Excel containing all the configuration & mapping
------------------------------------------------------------------------------------------
******************************************************************************************
Choose source by number
------------------------------------------------------------------------------------------
[1]  ~$conversion.xlsx	[2]  conversion.xlsx	[3]  conversion - Copy.xlsx	




Please choose Q(uit) or between 1 and 3:  2


******************************************************************************************
All done, the output (cSV and Excel) can be found timestamped in: ./Converted
------------------------------------------------------------------------------------------
